In [1]:
from typing import OrderedDict
import syft as sy
from syft.core.node.common.node_service.tff.tff_messages import TFFMessageWithReply

import os, sys

import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff    
import io
from tensorflow_federated.python.learning.model_utils import ModelWeights 


# np.random.seed(0)

/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
domain = sy.login(email="sam@stargate.net", password="changethis", port=8081)


Anyone can login into your account right now because your password is the default PySyft password!!!

Connecting to localhost... done! 	 Logging into affectionate_kellis... done!


In [3]:
domain.datasets



Idx,Name,Description,Assets,Id
[0],Mixed MedNIST 64 fo real,Small dataset for TFF testing,"[""images""] -> [""labels""] ->",e5cfb5b4-39ef-416c-8bae-0fa632a0c66f


In [4]:
def new_create_keras_model():
  return tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(64,64,1), name='input'),
        tf.keras.layers.Conv2D(64, kernel_size=(5, 5), activation="relu", name='conv1'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2), name='pool1'),
      #   tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu", name='conv2'),
      #   tf.keras.layers.MaxPooling2D(pool_size=(2, 2), name='pool2'),
      #   tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu", name='conv3'),
      #   tf.keras.layers.MaxPooling2D(pool_size=(2, 2), name='pool3'),
      #   tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation="relu", name='conv4'),
      #   tf.keras.layers.MaxPooling2D(pool_size=(2, 2), name='pool4'),
        tf.keras.layers.Flatten(),
        # tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(32, kernel_initializer='zeros'),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Dense(6, kernel_initializer='zeros'),
        tf.keras.layers.Softmax(),
  ])
  
def create_keras_model():
      return tf.keras.models.Sequential([
            tf.keras.layers.InputLayer(input_shape=(4096,)),
            tf.keras.layers.Dense(10, kernel_initializer='zeros'),
            tf.keras.layers.Softmax(),
      ])
  

In [6]:
model_fn = new_create_keras_model 
params = {
    'rounds': 1,
    'no_clients': 5,
    'noise_multiplier': 0.05,
    'clients_per_round': 2,
    'dataset_id': domain.datasets[0].id,
    'train_data_id': domain.datasets[0]['images'].id_at_location.to_string(),
    'label_data_id': domain.datasets[0]['labels'].id_at_location.to_string()
    }
 
model, metrics = sy.tff.train_model(model_fn, params, domain)

[1, 64, 64]
[1, 1]


INFO:tensorflow:Assets written to: _tmp/assets


INFO:tensorflow:Assets written to: _tmp/assets


In [7]:
metrics

[OrderedDict([('distributor', ()),
              ('client_work',
               OrderedDict([('train', OrderedDict([('loss', 0.5264447)]))])),
              ('aggregator',
               OrderedDict([('zeroing',
                             OrderedDict([('dp_query_metrics',
                                           OrderedDict([('clip', 0.1105171)])),
                                          ('dp', ())])),
                            ('zeroing_norm', 21.0),
                            ('zeroed_count', 0)])),
              ('finalizer', ())])]

In [8]:
model.layers

In [9]:
# download MedNIST.pkl
if not os.path.exists("./MedNIST.pkl"):
    os.system('curl -O "https://media.githubusercontent.com/media/shubham3121/datasets/main/MedNIST/MedNIST.pkl"')
else:
    print("MedNIST already downloaded")

MedNIST already downloaded


In [10]:
# Helper Methods

import os
import json
import pandas as pd
from PIL import Image
from enum import Enum
from collections import defaultdict
import numpy as np
from syft.core.adp.data_subject_list import DataSubjectList


def get_label_mapping():
    # the data uses the following mapping
    mapping = {
        "AbdomenCT": 0, 
        "BreastMRI": 1, 
        "CXR": 2, 
        "ChestCT": 3, 
        "Hand": 4, 
        "HeadCT": 5
    }
    return mapping

df = pd.read_pickle("./MedNIST.pkl")
mapping = get_label_mapping()

total_num = df.shape[0]
print("Columns:", df.columns)
print("Total Images:", total_num)
print("Label Mapping", mapping)

Columns: Index(['patient_id', 'image', 'label'], dtype='object')
Total Images: 58954
Label Mapping {'AbdomenCT': 0, 'BreastMRI': 1, 'CXR': 2, 'ChestCT': 3, 'Hand': 4, 'HeadCT': 5}


In [17]:
x = df['image'][10000]
x = tf.reshape(x, [-1,64,64])
x.shape


TensorShape([1, 64, 64])

In [18]:
model.predict(x)

1/1 [==============================] - 2s 2s/step


array([[0.14163701, 0.15815644, 0.14699148, 0.16503164, 0.17065217,
        0.21753128]], dtype=float32)

In [ ]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True